# Simulating simple models
In this document, the perfect integrator model, the drift diffusion absorbing model and the drift diffusion reflecting model will be simulated and the parameters of each one will be changed in order to understand the role of each variable.

In [1]:
# First, the libraries to be used will be imported:
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [2]:
# Then, the variables that will not change throughout the simulation will be defined:
sigma_i = 0 # I consider that there is no internal noise
xi_i = 0
tau = 200 #ms
tf = tau/40 #ms

## Perfect Integrator
In the first simulation the mu value will be varied from the start mu value to the final mu value, and the error of each mu value will be computed (for a thousand iterations):

In [3]:
# Defining the values used for this simulation
sigma_s = 0.25
sigma_i = 0.2
mu = 0
step = 0.1
X = []
Y = []
error_list = []
for z in range(999):
    # The starting decision value will be 0:
    x = 0
    decision_value = []
    iterations = 0
    xi_s = np.random.randn(1)
    stimulus = []
    counter = 0
    for i in range(199):
        stimulus.append(xi_s)
        counter += 1
        if counter == 10:
            xi_s  = np.random.randn(1)
            counter = 0
    # The process will be repeated a two hundred times (1 second stimulus)
    for i in range(199):
        # Normal random distribution noise:
        iterations += 1
        if iterations == 10:
            xi_s = np.random.randn(1)
            iterations = 0
        if xi_s> 0:
            stimulus.append(0)
        xi_i = np.random.randn(1)
        # Solved differential diffusion equation:
        x = x - (tf/tau)*(-mu)+np.sqrt(tf/tau)*(sigma_i*xi_i+sigma_s*xi_s)
        # All x will be stored in the deccision value list:
        if x> 0:
            decision_value.append(1)
        else:
            decision_value.append(0)
    stimulus_values.append(stimulus)
    decision_values.append(decision_value[len(decision_value)-1])
logreg = LogisticRegression()
logreg.fit(stimulus_values,decision_values)
y_pred=logreg.predict(decision_values)
#logit_mod = sm.Logit(X,Y)
#result = logit_mod.fit()
#result.params

NameError: name 'stimulus_values' is not defined

In [ ]:
plt.plot(np.arange(start_mu, final_mu, step),per_error_list)
plt.ylabel("Accuracy (%)")
plt.xlabel("mu value")
plt.title("Accuracy over mu value for the perfect integrator model")

Then mu will be set to a fixed value, and the sigma will be varied. The error percentage will be computed for each sigma (for 1000 iterations).

In [ ]:
# Defining the values used for this simulation
mu = 0.1
start_sigma_s = 0
final_sigma_s = 0.5
step = 0.1
per_error_list = []
# The mu value will be varied from the start mu value to the final mu value, separated by the step value:
for sigma_s in np.arange(start_sigma_s, final_sigma_s, step):
    for z in range(999):
        # The starting decision value will be 0:
        x = 0
        decision_value = [x]
        # The process will be repeated a two hundred times (1 second stimulus)
        for i in range(199):
            # Normal random distribution noise:
            xi_s = np.random.randn(1)
            # Solved differential diffusion equation:
            x = x - (tf/tau)*(-mu)+np.sqrt(tf/tau)*(sigma_i*xi_i+sigma_s*xi_s)
            # All x will be stored in the deccision value list:
            decision_value.append(x)

        # The boolean value of  the decision veing positive or negative will be calculated: 
        positive_decision = decision_value[len(decision_value)-1][0]>=0
        negative_decision = decision_value[len(decision_value)-1][0]<0
        positive_mu = mu>=0
        negative_mu = mu<0

        # If the last decision value and the mu have the same sign, there will be no error (0 will be stored), if they are different there will be error (1 will be stored)
        if positive_decision == positive_mu or negative_decision  == negative_mu:
            error_list.append(0)
        else:
            error_list.append(1)
        if z == 1:
            # Plotting the decision value over time:
            plt.plot(np.arange(0,200,1),decision_value,label = round(sigma_s,2))           
            plt.title("Perfect Integrator changing the sigma s value")
            plt.xlabel("Time (ms)")
            positions = [200/8*n for n in range(9)]
            labels = [1000/8*n for n in range(9)]
            plt.xticks(positions, labels)
            plt.ylabel("Decision value")
            plt.legend()
    # Printing the error for each mu
    per_error = sum(error_list)/len(error_list)*100
    per_error_list.append(100-per_error)
    #print("The incorrect decision is made", round(per_error,2), "% of the times for the sigma s value = ", round(sigma_s,3))


In [ ]:
plt.plot(np.arange(start_sigma_s, final_sigma_s, step),per_error_list)
plt.ylabel("Accuracy (%)")
plt.xlabel("sigma_s value")
plt.title("Accuracy over sigma_s value for the perfect integrator model")

## DDM Absorbing
In the first simulation the mu value will be varied from the start mu value to the final mu value, and the error of each mu value will be computed (for a thousand iterations):

In [ ]:
# Defining the values used for this simulation
sigma_s = 0.25
mu = 0.1
bound = 0.5
bound1 = -0.5


# The starting decision value will be 0:
x = 0
decision_value = [x]
# Starting with the condition that the bound hasn't been reached
reached_bound = False
# While the bound hasn't been reached, the equation will be performed over time       
while reached_bound == False:
    # Normal random distribution noise:
    xi_s = np.random.randn(1)
    # Solved differential diffusion equation:
    x = x - (tf/tau)*(-mu)+np.sqrt(tf/tau)*(sigma_i*xi_i+sigma_s*xi_s)
    # When the bound is reached, the rest of the decision value will be the bound value
    if x >= bound:
        if len(decision_value)<200:
            reached_bound = True
            for i in range(199-len(decision_value)):
                decision_value.append(bound)
        # When the iterations are over 200, break the loop:
        else:
            break
    if x <= bound1:
        if len(decision_value)<200:
            reached_bound = True
            for i in range(200-len(decision_value)):
                decision_value.append(bound1)
        # When the iterations are over 200, break the loop:
        else:
            break
    else:
        # All x before reaching the bound value will be stored in the deccision value list:
        decision_value.append(x)
    # If the bound hasn't been reached and its over 200 iterations, break the loop
    if len(decision_value)==200:
        break
plt.plot(np.arange(0,200,1),decision_value,"r")           
plt.title("DDM Absorbing")
plt.xlabel("Time (ms)")
positions = [200/8*n for n in range(9)]
labels = [1000/8*n for n in range(9)]
plt.xticks(positions, labels)
plt.ylabel("Decision value")
plt.axis([0,999/8,-0.6,0.6])

In [ ]:
# Defining the values used for this simulation
sigma_s = 0.25
start_mu = -0.5
final_mu = 0.6
step = 0.1
bound = 0.5
bound1 = -0.5
per_error_list = []

# The mu value will be varied from the start mu value to the final mu value, separated by the step value:
for mu in np.arange(start_mu, final_mu, step):
    # In this error_list, 0 will be stored if the sign of the mu value and the decision making are equal, and 1 will be stored if they are different:
    error_list = []
    for z in range(999):
        # The starting decision value will be 0:
        x = 0
        decision_value = [x]
        # Starting with the condition that the bound hasn't been reached
        reached_bound = False
        # While the bound hasn't been reached, the equation will be performed over time       
        while reached_bound == False:
            # Normal random distribution noise:
            xi_s = np.random.randn(1)
            # Solved differential diffusion equation:
            x = x - (tf/tau)*(-mu)+np.sqrt(tf/tau)*(sigma_i*xi_i+sigma_s*xi_s)
            # When the bound is reached, the rest of the decision value will be the bound value
            if x >= bound:
                if len(decision_value)<200:
                    reached_bound = True
                    for i in range(199-len(decision_value)):
                        decision_value.append(bound)
                # When the iterations are over 200, break the loop:
                else:
                    break
            if x <= bound1:
                if len(decision_value)<200:
                    reached_bound = True
                    for i in range(200-len(decision_value)):
                        decision_value.append(bound1)
                # When the iterations are over 200, break the loop:
                else:
                    break
            else:
                # All x before reaching the bound value will be stored in the deccision value list:
                decision_value.append(x)
            # If the bound hasn't been reached and its over 200 iterations, break the loop
            if len(decision_value)==200:
                break
        
        # The boolean value of  the decision veing positive or negative will be calculated: 
        positive_decision = decision_value[len(decision_value)-1]>=0
        negative_decision = decision_value[len(decision_value)-1]<0
        positive_mu = mu>=0
        negative_mu = mu<0

        # If the last decision value and the mu have the same sign, there will be no error (0 will be stored), if they are different there will be error (1 will be stored)
        if positive_decision == positive_mu or negative_decision  == negative_mu:
            error_list.append(0)
        else:
            error_list.append(1)
        # Only for one iteration the decision value will be plotted
        if z == 1:
            # Plotting the decision value over time:
            plt.plot(np.arange(0,200,1),decision_value,label = round(mu,2))           
            plt.title("DDM Absorbing changing the mu value")
            plt.xlabel("Time (ms)")
            positions = [200/8*n for n in range(9)]
            labels = [1000/8*n for n in range(9)]
            plt.xticks(positions, labels)
            plt.ylabel("Decision value")
            plt.legend()
            plt.axis([0,999/8,-0.6,0.6])
    per_error = sum(error_list)/len(error_list)*100
    per_error_list.append(100-per_error)
    #print("The incorrect decision is made", round(per_error,2), "% of the times for the mu value = ", round(mu,3))

In [ ]:
plt.plot(np.arange(start_mu, final_mu, step),per_error_list)
plt.ylabel("Accuracy (%)")
plt.xlabel("mu value")
plt.title("Accuracy over mu value for the DDM Absorbing")

Then mu will be set to a fixed value, and the sigma will be varied. The error percentage will be computed for each sigma (for 1000 iterations).

In [ ]:
# Defining the values used for this simulation
mu = 0.1
start_sigma_s = 0
final_sigma_s = 0.5
step = 0.1
bound = 0.5
bound1 = -0.5
per_error_list = []
# The mu value will be varied from the start mu value to the final mu value, separated by the step value:
for sigma_s in np.arange(start_sigma_s, final_sigma_s, step):
    # In this error_list, 0 will be stored if the sign of the mu value and the decision making are equal, and 1 will be stored if they are different:
    error_list = []
    for z in range(999):
        # The starting decision value will be 0:
        x = 0
        decision_value = [x]
        reached_bound = False
        # The process will be repeated two hundred times (1 second stimulus)
        
        while reached_bound == False:
            # Normal random distribution noise:
            xi_s = np.random.randn(1)
            # Solved differential diffusion equation:
            x = x - (tf/tau)*(-mu)+np.sqrt(tf/tau)*(sigma_i*xi_i+sigma_s*xi_s)
            # If the bound is reached:
            if x >= bound:
                if len(decision_value)<200:
                    reached_bound = True
                    for i in range(199-len(decision_value)):
                        decision_value.append(bound)
                # When iterations are 200:
                else:
                    break
            if x <= bound1:
                if len(decision_value)<200:
                    reached_bound = True
                    for i in range(200-len(decision_value)):
                        decision_value.append(bound1)
            else:
                # All x before reaching the bound value will be stored in the deccision value list:
                decision_value.append(x)
            # When iterations are 200:
            if len(decision_value)==200:
                break
        
        # The boolean value of  the decision veing positive or negative will be calculated: 
        positive_decision = decision_value[len(decision_value)-1]>=0
        negative_decision = decision_value[len(decision_value)-1]<0
        positive_mu = mu>=0
        negative_mu = mu<0

        # If the last decision value and the mu have the same sign, there will be no error (0 will be stored), if they are different there will be error (1 will be stored)
        if positive_decision == positive_mu or negative_decision  == negative_mu:
            error_list.append(0)
        else:
            error_list.append(1)
        if z == 1:
            # Plotting the decision value over time:
            plt.plot(np.arange(0,200,1),decision_value,label = round(sigma_s,2))           
            plt.title("DDM Absorbing changing de sigma s value")
            plt.xlabel("Time (ms)")
            positions = [200/8*n for n in range(9)]
            labels = [1000/8*n for n in range(9)]
            plt.xticks(positions, labels)
            plt.ylabel("Decision value")
            plt.legend()
            plt.axis([0,999/8,-0.6,0.6])
    per_error = sum(error_list)/len(error_list)*100
    per_error_list.append(100-per_error)
    #print("The incorrect decision is made", round(per_error,2), "% of the times for the sigma s value = ", round(sigma_s,3))

In [ ]:
plt.plot(np.arange(start_sigma_s, final_sigma_s, step),per_error_list)
plt.ylabel("Accuracy (%)")
plt.xlabel("sigma_s value")
plt.title("Accuracy over sigma_s value for the DDM Absorbing model")

## DDM reflecting
In the first simulation the mu value will be varied from the start mu value to the final mu value, and the error of each mu value will be computed (for a thousand iterations):

In [ ]:
# Defining the values used for this simulation
sigma_s = 0.25
mu = 0.1
# The starting decision value will be 0:
x = 0
decision_value = [x]
# The process will be repeated two hundred times (1 second stimulus)
for i in range(199):
    # Normal random distribution noise:
    xi_s = np.random.randn(1)
    # Solved differential diffusion equation:
    x = x - (tf/tau)*(-mu)+np.sqrt(tf/tau)*(sigma_i*xi_i+sigma_s*xi_s)
    # If the bound is exceeded, it will return the value of the bound
    if x> bound:
        decision_value.append(bound)
    elif x < bound1:
        decision_value.append(bound1)
    else:
        decision_value.append(x)
# Plotting the decision value over time:
plt.plot(np.arange(0,200,1),decision_value,"r")           
plt.title("DDM Reflecting")
plt.xlabel("Time (ms)")
positions = [200/8*n for n in range(9)]
labels = [1000/8*n for n in range(9)]
plt.xticks(positions, labels)
plt.ylabel("Decision value")


In [ ]:
# Defining the values used for this simulation
sigma_s = 0.25
start_mu = -0.5
final_mu = 0.6
step = 0.1
per_error_list = []
# The mu value will be varied from the start mu value to the final mu value, separated by the step value:
for mu in np.arange(start_mu, final_mu, step):
    # In this error_list, 0 will be stored if the sign of the mu value and the decision making are equal, and 1 will be stored if they are different:
    error_list = []
    for z in range(999):
        # The starting decision value will be 0:
        x = 0
        decision_value = [x]
        # The process will be repeated two hundred times (1 second stimulus)
        for i in range(199):
            # Normal random distribution noise:
            xi_s = np.random.randn(1)
            # Solved differential diffusion equation:
            x = x - (tf/tau)*(-mu)+np.sqrt(tf/tau)*(sigma_i*xi_i+sigma_s*xi_s)
            # If the bound is exceeded, it will return the value of the bound
            if x> bound:
                decision_value.append(bound)
            elif x < bound1:
                decision_value.append(bound1)
            else:
                decision_value.append(x)
        
        # The boolean value of  the decision veing positive or negative will be calculated: 
        positive_decision = decision_value[len(decision_value)-1]>=0
        negative_decision = decision_value[len(decision_value)-1]<0
        positive_mu = mu>=0
        negative_mu = mu<0

        # If the last decision value and the mu have the same sign, there will be no error (0 will be stored), if they are different there will be error (1 will be stored)
        if positive_decision == positive_mu or negative_decision  == negative_mu:
            error_list.append(0)
        else:
            error_list.append(1)
        # Only for one iteration
        if z == 1:
            # Plotting the decision value over time:
            plt.plot(np.arange(0,200,1),decision_value,label = round(mu,2))           
            plt.title("DDM reflecting model changing the mu value")
            plt.xlabel("Time (ms)")
            positions = [200/8*n for n in range(9)]
            labels = [1000/8*n for n in range(9)]
            plt.xticks(positions, labels)
            plt.ylabel("Decision value")
            plt.legend()
    per_error = sum(error_list)/len(error_list)*100
    per_error_list.append(100-per_error)
    #print("The incorrect decision is made", round(per_error,2), "% of the times for the mu value = ", round(mu,3))

In [ ]:
plt.plot(np.arange(start_mu, final_mu, step),per_error_list)
plt.ylabel("Accuracy (%)")
plt.xlabel("mu value")
plt.title("Accuracy over mu value for the DDM Reflecting")

Then mu will be set to a fixed value, and the sigma will be varied. The error percentage will be computed for each sigma (for 1000 iterations).

In [ ]:
# Defining the values used for this simulation
mu = 0.1
start_sigma_s = 0
final_sigma_s = 0.5
step = 0.1
bound = 0.5
bound1 = -0.5
per_error_list = []
# The mu value will be varied from the start mu value to the final mu value, separated by the step value:
for sigma_s in np.arange(start_sigma_s, final_sigma_s, step):
    # In this error_list, 0 will be stored if the sign of the mu value and the decision making are equal, and 1 will be stored if they are different:
    error_list = []
    for z in range(999):
        # The starting decision value will be 0:
        x = 0
        decision_value = [x]
        # The process will be repeated two hundred times (1 second stimulus)
        for i in range(199):
            # Normal random distribution noise:
            xi_s = np.random.randn(1)
            # Solved differential diffusion equation:
            x = x - (tf/tau)*(-mu)+np.sqrt(tf/tau)*(sigma_i*xi_i+sigma_s*xi_s)
            # If the bound is exceeded it will return the value of the bound
            if x> bound:
                decision_value.append(bound)
            elif x < bound1:
                decision_value.append(bound1)
            else:
                decision_value.append(x)
        
        # The boolean value of  the decision veing positive or negative will be calculated: 
        positive_decision = decision_value[len(decision_value)-1]>=0
        negative_decision = decision_value[len(decision_value)-1]<0
        positive_mu = mu>=0
        negative_mu = mu<0

        # If the last decision value and the mu have the same sign, there will be no error (0 will be stored), if they are different there will be error (1 will be stored)
        if positive_decision == positive_mu or negative_decision  == negative_mu:
            error_list.append(0)
        else:
            error_list.append(1)
        # Only for one itration
        if z == 1:
            # Plotting the decision value over time:
            plt.plot(np.arange(0,200,1),decision_value,label = round(sigma_s,2))           
            plt.title("DDM Reflecting changing the sigma_s value")
            plt.xlabel("Time (ms)")
            positions = [200/8*n for n in range(9)]
            labels = [1000/8*n for n in range(9)]
            plt.xticks(positions, labels)
            plt.ylabel("Decision value")
            plt.legend()
    per_error = sum(error_list)/len(error_list)*100
    per_error_list.append(100-per_error)
    #print("The incorrect decision is made", round(per_error,2), "% of the times for the sigma s value = ", round(sigma_s,3))

In [ ]:
plt.plot(np.arange(start_sigma_s, final_sigma_s, step),per_error_list)
plt.ylabel("Accuracy (%)")
plt.xlabel("sigma_s value")
plt.title("Accuracy over sigma_s value for the DDM Reflecting model")